In [1]:
from time import sleep
from datetime import datetime, timezone, timedelta
from alpaca_api.alpaca_api import AlpacaApi

from database.market import Market
from processor.processor import Processor as p
from modeler_strats.universal_modeler import UniversalModeler
from datetime import datetime
from fund.fund import Fund

from pricer.pricer import Pricer as pricer_list
from ranker.ranker import Ranker as ranker_list
from classifier.classifier import Classifier as classifier_list
from portfolio.aportfolio import APortfolio
import pandas as pd

from returns.products import Products
from processor.processor import Processor as p
from time import sleep

import json

In [2]:
market = Market()

In [15]:
parameter = {
    "value":True
    ,"ceiling":True
    ,"classification":False
    ,"rank":False
    ,"short":False
    ,"risk":True
    ,"market_return":1.15
    ,"buy_day":1
    ,"sell_day":5
}

In [16]:
positions = 20
deploy = True
alp = AlpacaApi()


start = (datetime.now() - timedelta(days=365)).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")

portfolio = APortfolio(pricer_list.WEEKLY_STOCK_SPECULATION
                          ,classifier_list.NONE
                          ,ranker_list.NONE)
portfolios = [portfolio]
fund = Fund(portfolios,start,end,end)
fund.initialize_portfolios()
fund.initialize_backtesters()
modeler_strat = UniversalModeler()


day_to_deploy = parameter["buy_day"]
hour_to_deploy = 11

try:
    new_york_date = datetime.now(tz=timezone(offset=timedelta(hours=-4)))
    year = new_york_date.year
    week = new_york_date.isocalendar()[1]
#     hour = new_york_date.hour
#     day = new_york_date.weekday()
#     if day == day_to_deploy and hour == hour_to_deploy:
        ## Market Ops
    closed_orders = alp.paper_close_all()
    closed_order_df = pd.DataFrame([json.loads(closed_order.json())["body"] for closed_order in closed_orders])
    if closed_order_df.index.size > 1:
        portfolio.db.cloud_connect()
        portfolio.db.store("orders",closed_order_df)
        portfolio.db.disconnect()
    market.cloud_connect()
    sp500 = market.retrieve("sp500").rename(columns={"Symbol":"ticker"})
    tyields = Products.tyields(market.retrieve("tyields"))
    bench = Products.spy_bench(market.retrieve("spy"))

    ## creating prediction data
    portfolio = fund.portfolios[0]
    pricer_class = portfolio.pricer_class
    training_sets = []
    for ticker in sp500["ticker"].unique():
        try:
            prices = market.retrieve_ticker_prices(pricer_class.asset_class.value,ticker)
            prices = p.column_date_processing(prices)
            ticker_data = pricer_class.training_set(ticker,prices,True)
            training_sets.append(ticker_data)
        except Exception as e:
            print(ticker,str(e))
            continue
    data = pd.concat(training_sets)
    training_data = data.dropna().copy().sort_values(["year","week"])
    market.disconnect()

    # making predictions
    pricer_class.db.cloud_connect()
    pricer_class.db.drop("predictions")
    prediction_set = training_data[training_data["year"]==year].reset_index(drop=True)
    models = pricer_class.db.retrieve("models")
    stuff = modeler_strat.recommend(models,prediction_set,pricer_class.factors)
    stuff = stuff.rename(columns={"prediction":f"price_prediction"})
    stuff = pricer_class.sim_processor(stuff)
    relevant_columns = list(set(list(stuff.columns)) - set(pricer_class.factors))
    pricer_class.db.store("predictions",stuff[relevant_columns])
    pricer_class.db.disconnect()

    # making recs
    market.cloud_connect()
    returns = portfolio.create_current_returns(market,bench)
    market.disconnect()

    sim = stuff[relevant_columns].copy().drop("adjclose",axis=1)
    merged = portfolio.merge_sim_returns(sim,returns)
    merged = merged.sort_values(["year","week","day"]).dropna()
    stuff = portfolio.backtester.recommendation(merged.copy(),parameter,tyields)

    rec = stuff[0]
    rec_filtered = rec[(rec["week"]==week)]
    trades = rec_filtered.head(positions).merge(sp500[["ticker","Security","GICS Sector"]],on="ticker")
    final = trades[["year","week","ticker","Security","GICS Sector","weekly_delta","weekly_delta_sign"]]

    if final.index.size > 0:
        portfolio.db.cloud_connect()
        portfolio.db.store("recs",final)
        portfolio.db.disconnect()
        account = alp.paper_get_account()
        cash = float(account.cash) - 10
        # executing order
        order_data = []
        for row in trades.iterrows():
            try:
                ticker = row[1]["ticker"]
                amount = round(cash / positions) 
                order_data.append(alp.paper_market_order(ticker,amount))
                sleep(1)
            except Exception as e:
                portfolio.db.cloud_connect()
                portfolio.db.store("errors",pd.DataFrame([{"date":str(datetime.now()),"error":str(e)}]))
                portfolio.db.disconnect()
        order_dicts = pd.DataFrame([json.loads(order_d.json()) for order_d in order_data])
        portfolio.db.cloud_connect()
        portfolio.db.store("orders",order_dicts)
        portfolio.db.disconnect()

    portfolio.db.cloud_connect()
    portfolio.db.store("iterations",pd.DataFrame([{"date":str(datetime.now()),"status":"complete"}]))
    portfolio.db.disconnect()
    
except Exception as e:
        portfolio.db.cloud_connect()
        portfolio.db.store("iterations",pd.DataFrame([{"date":str(datetime.now()),"status":"incomplete"}]))
        portfolio.db.store("errors",pd.DataFrame([{"date":str(datetime.now()),"error":str(e)}]))
        portfolio.db.disconnect()

In [17]:
final_data = merged.copy()
market_return = parameter["market_return"]
final_data = final_data[final_data["day"]==parameter["buy_day"]-1]
final_data = portfolio.returns.returns(market_return,portfolio.pricer_class.time_horizon_class,final_data.copy(),True,tyields)

In [18]:
final_data

,year,week,day,ticker,cat_prediction,xgb_score,xgb_prediction,cat_score,price_prediction,close,...,bench_weekly_return,weekly_variance,market_weekly_cov,weekly_beta,weekly_yield,market_weekly_return,projected_weekly_return,weekly_delta,weekly_delta_sign,weekly_rrr
0,2023,5,1.0,A,156.026947,0.998962,158.174164,0.940958,157.100555,152.08,...,0.022536,0.000351,-0.000046,-0.130047,1.000881,1.002691,0.034699,0.034699,1.0,0.000646
1,2023,5,1.0,AAL,15.479273,0.998962,16.539669,0.940958,16.009471,16.14,...,0.022536,0.000351,0.000519,1.479722,1.000881,1.002691,-0.008087,0.008087,-1.0,0.003560
2,2023,5,1.0,AAP,145.108303,0.998962,149.734024,0.940958,147.421163,152.28,...,0.022536,0.000351,-0.000151,-0.431771,1.000881,1.002691,-0.020125,0.020125,-1.0,0.000099
3,2023,5,1.0,AAPL,141.152218,0.998962,144.500961,0.940958,142.826590,144.29,...,0.022536,0.000351,-0.000110,-0.312607,1.000881,1.002691,-0.008635,0.008635,-1.0,0.000315
4,2023,5,1.0,ABBV,143.587218,0.998962,147.322449,0.940958,145.454833,147.75,...,0.022536,0.000351,-0.000137,-0.391541,1.000881,1.002691,-0.006578,0.006578,-1.0,0.000172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10236,2023,25,1.0,YUM,136.574778,0.998962,137.772263,0.940958,137.173520,135.82,...,0.023252,0.000107,0.000119,1.114770,1.000981,1.002691,0.009966,0.009966,1.0,0.002888
10237,2023,25,1.0,ZBH,139.378927,0.998962,140.215820,0.940958,139.797374,143.66,...,0.023252,0.000107,0.000215,2.008718,1.000981,1.002691,-0.026887,0.026887,-1.0,0.004416
10238,2023,25,1.0,ZBRA,282.490068,0.998962,285.800964,0.940958,284.145516,277.55,...,0.023252,0.000107,0.000130,1.211751,1.000981,1.002691,0.023763,0.023763,1.0,0.003053
10239,2023,25,1.0,ZION,27.726021,0.998962,29.195671,0.940958,28.460846,27.85,...,0.023252,0.000107,0.000063,0.592867,1.000981,1.002691,0.021933,0.021933,1.0,0.001995
